In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

# Matplotlib settings
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


In [155]:
# Cell 2: Set Your File Paths
base_path = r"C:\Users\dmath\Downloads\nfl-big-data-bowl-2026-analytics\114239_nfl_competition_files_published_analytics_final"
supp_path = os.path.join(base_path, "supplementary_data.csv")
train_folder = os.path.join(base_path, "train")

print(f"✅ Base path set: {base_path}")
print(f"✅ Supplementary data: {supp_path}")
print(f"✅ Training folder: {train_folder}")


✅ Base path set: C:\Users\dmath\Downloads\nfl-big-data-bowl-2026-analytics\114239_nfl_competition_files_published_analytics_final
✅ Supplementary data: C:\Users\dmath\Downloads\nfl-big-data-bowl-2026-analytics\114239_nfl_competition_files_published_analytics_final\supplementary_data.csv
✅ Training folder: C:\Users\dmath\Downloads\nfl-big-data-bowl-2026-analytics\114239_nfl_competition_files_published_analytics_final\train


In [156]:
# Cell 3: Load All Data Files
print("⏳ Loading supplementary data...")
supp_df = pd.read_csv(supp_path)
print(f"✅ Loaded supplementary data: {supp_df.shape}")

print("\n⏳ Loading input tracking data (18 weeks)...")
input_files = [os.path.join(train_folder, f"input_2023_w{str(i).zfill(2)}.csv") for i in range(1, 19)]
input_df = pd.concat([pd.read_csv(f) for f in input_files], ignore_index=True)
print(f"✅ Loaded input data: {input_df.shape}")

print("\n⏳ Loading output tracking data (18 weeks)...")
output_files = [os.path.join(train_folder, f"output_2023_w{str(i).zfill(2)}.csv") for i in range(1, 19)]
output_df = pd.concat([pd.read_csv(f) for f in output_files], ignore_index=True)
print(f"✅ Loaded output data: {output_df.shape}")

print("\n✅ ALL DATA LOADED SUCCESSFULLY!")


⏳ Loading supplementary data...
✅ Loaded supplementary data: (18009, 41)

⏳ Loading input tracking data (18 weeks)...


C:\Users\dmath\AppData\Local\Temp\ipykernel_25036\2491330242.py:3: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  supp_df = pd.read_csv(supp_path)


✅ Loaded input data: (4880579, 23)

⏳ Loading output tracking data (18 weeks)...
✅ Loaded output data: (562936, 6)

✅ ALL DATA LOADED SUCCESSFULLY!


In [157]:
# Cell 4: Merge and Prepare Red Zone Data

print("⏳ Merging tracking data with play context...")
merged_df = pd.merge(input_df, supp_df, on=['game_id', 'play_id'], how='inner')
print(f"✅ Merged data: {merged_df.shape}")

# Define red zone interval function
def get_redzone_interval(yardline):
    """Categorize plays into red zone intervals"""
    if 5 <= yardline <= 10:
        return '5-10'
    elif 10 < yardline <= 15:
        return '10-15'
    elif 15 < yardline <= 20:
        return '15-20'
    else:
        return None

print("\n⏳ Filtering for red zone plays...")
merged_df['redzone_interval'] = merged_df['yardline_number'].apply(get_redzone_interval)
redzone_df = merged_df[merged_df['redzone_interval'].notna()].copy()
print(f"✅ Red zone plays: {redzone_df['play_id'].nunique()} unique plays")

print("\n✅ DATA PREPROCESSING COMPLETE!")


⏳ Merging tracking data with play context...
✅ Merged data: (4880579, 62)

⏳ Filtering for red zone plays...
✅ Red zone plays: 1982 unique plays

✅ DATA PREPROCESSING COMPLETE!


In [158]:
# Cell 5: Aggregate Play-Level Statistics

print("⏳ Creating play-level summary...")
play_summary = redzone_df.groupby(['game_id', 'play_id']).agg({
    'redzone_interval': 'first',
    'possession_team': 'first',
    'play_description': 'first',
    'offense_formation': 'first',
    'receiver_alignment': 'first',
    'route_of_targeted_receiver': 'first',
    'pass_result': 'first',
    'play_action': 'first',
    'dropback_type': 'first',
    'team_coverage_man_zone': 'first',
    'team_coverage_type': 'first',
    'yards_gained': 'first',
    'down': 'first',
    'yards_to_go': 'first'
}).reset_index()

print("⏳ Identifying touchdown plays...")
play_summary['is_touchdown'] = play_summary['play_description'].str.contains('TOUCHDOWN', case=False, na=False)
successful_plays = play_summary[play_summary['is_touchdown'] == True].copy()

print(f"✅ Total plays analyzed: {len(play_summary)}")
print(f"✅ Touchdown plays: {len(successful_plays)}")
print(f"✅ Overall TD rate: {len(successful_plays)/len(play_summary)*100:.1f}%")

print("\n✅ PLAY SUMMARY COMPLETE!")


⏳ Creating play-level summary...
⏳ Identifying touchdown plays...
✅ Total plays analyzed: 2692
✅ Touchdown plays: 318
✅ Overall TD rate: 11.8%

✅ PLAY SUMMARY COMPLETE!


In [159]:
# Cell 6: Define Recommendation Engine

def get_recommendations(yards_out, defense_type):
    """
    Get play recommendations based on field position and expected defense
    
    Args:
        yards_out (int): Distance from endzone (5-20)
        defense_type (str): 'Man' or 'Zone'
    
    Returns:
        dict: Comprehensive recommendations with optimal execution parameters
    """
    # Determine interval
    if 5 <= yards_out <= 10:
        interval = '5-10'
    elif 10 < yards_out <= 15:
        interval = '10-15'
    elif 15 < yards_out <= 20:
        interval = '15-20'
    else:
        return {'error': 'Yards must be between 5-20'}
    
    # Filter for this scenario
    scenario_plays = successful_plays[
        (successful_plays['redzone_interval'] == interval) &
        (successful_plays['team_coverage_man_zone'] == defense_type)
    ]
    
    if len(scenario_plays) < 1:
        return {
            'error': f'Insufficient data for {interval} yards vs {defense_type} coverage',
            'available_plays': len(scenario_plays)
        }
    
    # Get play pattern recommendations
    patterns = scenario_plays.groupby([
        'offense_formation',
        'receiver_alignment',
        'route_of_targeted_receiver'
    ]).size().reset_index(name='td_count')
    
    patterns = patterns.sort_values('td_count', ascending=False)
    
    # Calculate success rates
    total_attempts = play_summary[
        (play_summary['redzone_interval'] == interval) &
        (play_summary['team_coverage_man_zone'] == defense_type)
    ]
    
    recommendations = []
    
    for idx, pattern in patterns.head(5).iterrows():
        # Get tracking data for this pattern
        pattern_td_plays = scenario_plays[
            (scenario_plays['offense_formation'] == pattern['offense_formation']) &
            (scenario_plays['receiver_alignment'] == pattern['receiver_alignment']) &
            (scenario_plays['route_of_targeted_receiver'] == pattern['route_of_targeted_receiver'])
        ]
        
        pattern_all_plays = total_attempts[
            (total_attempts['offense_formation'] == pattern['offense_formation']) &
            (total_attempts['receiver_alignment'] == pattern['receiver_alignment']) &
            (total_attempts['route_of_targeted_receiver'] == pattern['route_of_targeted_receiver'])
        ]
        
        success_rate = (len(pattern_td_plays) / len(pattern_all_plays) * 100) if len(pattern_all_plays) > 0 else 0
        
        # Get player movement data
        pattern_tracking = redzone_df[
            redzone_df['play_id'].isin(pattern_td_plays['play_id'])
        ]
        
        # Receiver metrics
        receiver_data = pattern_tracking[pattern_tracking['player_role'] == 'Targeted Receiver']
        other_receivers = pattern_tracking[pattern_tracking['player_role'] == 'Other Route Runner']
        
        rec = {
            'rank': idx + 1,
            'formation': pattern['offense_formation'],
            'alignment': pattern['receiver_alignment'],
            'route': pattern['route_of_targeted_receiver'],
            'td_count': int(pattern['td_count']),
            'success_rate': round(success_rate, 1),
            'total_attempts': len(pattern_all_plays),
            
            # Targeted Receiver optimal parameters
            'receiver_avg_speed': round(receiver_data['s'].mean(), 2) if not receiver_data.empty else None,
            'receiver_max_speed': round(receiver_data['s'].quantile(0.75), 2) if not receiver_data.empty else None,
            'receiver_avg_accel': round(receiver_data['a'].mean(), 2) if not receiver_data.empty else None,
            'receiver_max_accel': round(receiver_data['a'].quantile(0.75), 2) if not receiver_data.empty else None,
            'receiver_starting_x': round(receiver_data['x'].mean(), 1) if not receiver_data.empty else None,
            'receiver_starting_y': round(receiver_data['y'].mean(), 1) if not receiver_data.empty else None,
            'receiver_position_variance_x': round(receiver_data['x'].std(), 1) if not receiver_data.empty else None,
            'receiver_position_variance_y': round(receiver_data['y'].std(), 1) if not receiver_data.empty else None,
            
            # Other receivers
            'other_receivers_avg_speed': round(other_receivers['s'].mean(), 2) if not other_receivers.empty else None,
            'other_receivers_starting_x': round(other_receivers['x'].mean(), 1) if not other_receivers.empty else None,
            'other_receivers_starting_y': round(other_receivers['y'].mean(), 1) if not other_receivers.empty else None,
            
            # Play action usage
            'play_action_pct': round(pattern_td_plays['play_action'].mean() * 100, 1) if 'play_action' in pattern_td_plays else None,
        }
        
        recommendations.append(rec)
    
    return {
        'scenario': {
            'yards_out': yards_out,
            'interval': interval,
            'defense': defense_type,
            'total_td_plays': len(scenario_plays),
            'overall_td_rate': round(len(scenario_plays) / len(total_attempts) * 100, 1) if len(total_attempts) > 0 else 0
        },
        'recommendations': recommendations
    }

print("✅ Recommendation engine function defined!")


✅ Recommendation engine function defined!


In [160]:
# Cell 7: Define Display Function

def display_recommendations(result):
    """Format and display recommendations in a clean, readable format"""
    
    if 'error' in result:
        display(HTML(f"""
        <div style='background-color: #ffcccc; padding: 20px; border-radius: 10px; border-left: 5px solid #cc0000;'>
            <h3 style='color: #cc0000; margin: 0;'>❌ {result['error']}</h3>
            {f"<p style='margin: 10px 0 0 0;'>Only {result['available_plays']} TD plays found for this scenario</p>" if 'available_plays' in result else ""}
        </div>
        """))
        return
    
    scenario = result['scenario']
    recs = result['recommendations']
    
    # Header
    display(HTML(f"""
    <div style='background: linear-gradient(135deg, #1f77b4 0%, #2ca02c 100%); padding: 30px; border-radius: 15px; margin-bottom: 20px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);'>
        <h1 style='color: white; margin: 0; font-size: 32px;'>🏈 NFL RED ZONE PLAY CALLER</h1>
        <p style='color: #e0e0e0; margin: 10px 0 0 0; font-size: 16px;'>Data-Driven Play Recommendations</p>
    </div>
    """))
    
    # Scenario Summary
    display(HTML(f"""
    <div style='background-color: #f0f8ff; padding: 20px; border-radius: 10px; margin-bottom: 20px; border-left: 5px solid #1f77b4;'>
        <h2 style='color: #1f77b4; margin: 0 0 15px 0;'>📍 GAME SITUATION</h2>
        <table style='width: 100%; border-collapse: collapse;'>
            <tr>
                <td style='padding: 8px; font-weight: bold; width: 40%;'>Distance from Endzone:</td>
                <td style='padding: 8px;'>{scenario['yards_out']} yards ({scenario['interval']} yard interval)</td>
            </tr>
            <tr style='background-color: #e6f2ff;'>
                <td style='padding: 8px; font-weight: bold;'>Expected Defense:</td>
                <td style='padding: 8px;'>{scenario['defense']} Coverage</td>
            </tr>
            <tr>
                <td style='padding: 8px; font-weight: bold;'>Historical Success:</td>
                <td style='padding: 8px;'>{scenario['total_td_plays']} TDs | {scenario['overall_td_rate']}% success rate</td>
            </tr>
        </table>
    </div>
    """))
    
    # Recommendations
    for rec in recs:
        gradient_colors = ['#2ecc71', '#3498db', '#9b59b6', '#e74c3c', '#f39c12']
        color = gradient_colors[min(rec['rank']-1, 4)]
        
        display(HTML(f"""
        <div style='background-color: white; padding: 25px; border-radius: 10px; margin-bottom: 20px; border-left: 5px solid {color}; box-shadow: 0 2px 4px rgba(0,0,0,0.1);'>
            <div style='display: flex; justify-content: space-between; align-items: center; margin-bottom: 15px;'>
                <h2 style='color: {color}; margin: 0;'>#{rec['rank']} RECOMMENDED PLAY</h2>
                <div style='background-color: {color}; color: white; padding: 10px 20px; border-radius: 25px; font-weight: bold;'>
                    {rec['success_rate']}% Success ({rec['td_count']}/{rec['total_attempts']} TDs)
                </div>
            </div>
            
            <div style='background-color: #f8f9fa; padding: 15px; border-radius: 8px; margin-bottom: 15px;'>
                <h3 style='color: #333; margin: 0 0 10px 0;'>📋 PLAY STRUCTURE</h3>
                <table style='width: 100%;'>
                    <tr>
                        <td style='padding: 5px; font-weight: bold; width: 30%;'>Formation:</td>
                        <td style='padding: 5px;'>{rec['formation']}</td>
                    </tr>
                    <tr>
                        <td style='padding: 5px; font-weight: bold;'>Receiver Alignment:</td>
                        <td style='padding: 5px;'>{rec['alignment']}</td>
                    </tr>
                    <tr>
                        <td style='padding: 5px; font-weight: bold;'>Primary Route:</td>
                        <td style='padding: 5px;'>{rec['route']}</td>
                    </tr>
                    {f"<tr><td style='padding: 5px; font-weight: bold;'>Play Action:</td><td style='padding: 5px;'>Used {rec['play_action_pct']}% of the time</td></tr>" if rec['play_action_pct'] else ""}
                </table>
            </div>
            
            <div style='background-color: #fff3cd; padding: 15px; border-radius: 8px;'>
                <h3 style='color: #856404; margin: 0 0 10px 0;'>🎯 TARGETED RECEIVER - OPTIMAL EXECUTION</h3>
                <table style='width: 100%;'>
                    {f"<tr><td style='padding: 5px; font-weight: bold; width: 35%;'>Starting Position:</td><td style='padding: 5px;'>({rec['receiver_starting_x']}, {rec['receiver_starting_y']}) yards</td></tr>" if rec['receiver_starting_x'] else ""}
                    {f"<tr><td style='padding: 5px; font-weight: bold;'>Position Flexibility:</td><td style='padding: 5px;'>±{rec['receiver_position_variance_x']} x-axis, ±{rec['receiver_position_variance_y']} y-axis</td></tr>" if rec['receiver_position_variance_x'] else ""}
                    {f"<tr><td style='padding: 5px; font-weight: bold;'>Pre-Snap Speed:</td><td style='padding: 5px;'>{rec['receiver_avg_speed']} yd/s (peak: {rec['receiver_max_speed']} yd/s)</td></tr>" if rec['receiver_avg_speed'] else ""}
                    {f"<tr><td style='padding: 5px; font-weight: bold;'>Acceleration:</td><td style='padding: 5px;'>{rec['receiver_avg_accel']} yd/s² (peak: {rec['receiver_max_accel']} yd/s²)</td></tr>" if rec['receiver_avg_accel'] else ""}
                </table>
            </div>
        </div>
        """))

print("✅ Display function defined!")


✅ Display function defined!


In [161]:
# Cell 14: Simulation Engine & Advanced Analytics

import scipy.stats as stats

# 1. ROBUST FILTERING HELPER
def map_coverage_input(user_input, available_coverages):
    """Maps simple 'Man'/'Zone' input to actual dataset values like 'MAN_COVERAGE'"""
    user_input = user_input.upper()
    for cov in available_coverages:
        if isinstance(cov, str):
            if user_input in cov: # e.g., "MAN" is inside "MAN_COVERAGE"
                return cov
    return user_input # Fallback

# 2. MONTE CARLO SIMULATION ENGINE
def simulate_play_reliability(successes, attempts, global_avg_rate=0.25, simulations=10000):
    """
    Simulates 10,000 seasons to determine the TRUE reliability of a play.
    Uses Bayesian inference (Beta-Binomial) to handle small sample sizes.
    
    Returns:
        expected_rate: The most likely true success rate (smoothed).
        lower_bound: The conservative estimate (90% confidence it's at least this good).
    """
    # Prior belief: A play is likely to be average (approx 25% in red zone)
    # We weight this prior as if we've seen 10 "average" plays.
    prior_alpha = global_avg_rate * 10
    prior_beta = (1 - global_avg_rate) * 10
    
    # Update with real data
    posterior_alpha = prior_alpha + successes
    posterior_beta = prior_beta + (attempts - successes)
    
    # Monte Carlo Simulation: Sample from the resulting probability distribution
    simulated_rates = stats.beta.rvs(posterior_alpha, posterior_beta, size=simulations)
    
    # Metrics
    expected_rate = np.mean(simulated_rates) * 100
    lower_bound_90 = np.percentile(simulated_rates, 10) * 100 # Conservative score
    
    return round(expected_rate, 1), round(lower_bound_90, 1)

print("✅ Simulation Engine & Robust Filter loaded!")


✅ Simulation Engine & Robust Filter loaded!


In [162]:
# Cell 15: Recommendation Engine with Reliability Sorting

def get_ranked_recommendations(yards_out, defense_type):
    # 1. Determine Interval
    if 5 <= yards_out <= 10:
        interval = '5-10'
    elif 10 < yards_out <= 15:
        interval = '10-15'
    elif 15 < yards_out <= 20:
        interval = '15-20'
    else:
        return {'error': 'Yards must be between 5-20'}

    # 2. Map Coverage Input to Data Values
    # Based on your screenshot, values are like "MAN_COVERAGE"
    available_covs = play_summary['team_coverage_man_zone'].unique()
    mapped_coverage = map_coverage_input(defense_type, available_covs)
    
    print(f"🔎 Analzying: {interval} yards vs {mapped_coverage}...")

    # 3. Filter Data
    scenario_plays = successful_plays[
        (successful_plays['redzone_interval'] == interval) &
        (successful_plays['team_coverage_man_zone'] == mapped_coverage)
    ]
    
    all_attempts_scenario = play_summary[
        (play_summary['redzone_interval'] == interval) &
        (play_summary['team_coverage_man_zone'] == mapped_coverage)
    ]
    
    if len(all_attempts_scenario) < 5:
         return {'error': f"Insufficient sample size ({len(all_attempts_scenario)} plays found)."}

    # 4. Group by Play Structure
    # We group by Formation + Route to aggregate sample sizes
    grouped = all_attempts_scenario.groupby(['offense_formation', 'route_of_targeted_receiver']).agg(
        total_attempts=('play_id', 'count'),
        td_count=('is_touchdown', 'sum')
    ).reset_index()

    # Filter out extremely rare plays (need at least 2 attempts to simulate meaningfully)
    grouped = grouped[grouped['total_attempts'] >= 2].copy()

    # 5. RUN SIMULATIONS & CALCULATE METRICS
    results = []
    
    # Calculate global average for Prior
    global_avg = len(successful_plays) / len(play_summary)
    
    for _, row in grouped.iterrows():
        # Run Monte Carlo Simulation
        expected_rate, reliability_score = simulate_play_reliability(
            row['td_count'], 
            row['total_attempts'], 
            global_avg_rate=global_avg
        )
        
        raw_rate = (row['td_count'] / row['total_attempts']) * 100
        
        results.append({
            'formation': row['offense_formation'],
            'route': row['route_of_targeted_receiver'],
            'td_count': row['td_count'],
            'attempts': row['total_attempts'],
            'raw_success_rate': round(raw_rate, 1),
            'simulated_success': expected_rate,     # The "Smoothed" Average
            'reliability_score': reliability_score  # The Conservative "Safe" Bet
        })

    # 6. SORTING LOGIC
    # Primary Sort: Reliability Score (Conservative estimate)
    # This prevents 1/1 (100%) from beating 8/10 (80%)
    df_results = pd.DataFrame(results)
    if df_results.empty:
        return {'error': "No recurring play patterns found."}
        
    df_results = df_results.sort_values('reliability_score', ascending=False)

    return {
        'scenario': {'interval': interval, 'defense': mapped_coverage},
        'data': df_results.head(5).to_dict('records') # Top 5
    }

print("✅ Ranked Recommendation Engine loaded!")


✅ Ranked Recommendation Engine loaded!


In [163]:
# Cell 16: Interface with Reliability Metrics

def display_ranked_results(result):
    if 'error' in result:
        print(f"❌ {result['error']}")
        return

    scenario = result['scenario']
    recs = result['data']
    
    display(HTML(f"""
    <div style='background: linear-gradient(to right, #1a2a6c, #b21f1f, #fdbb2d); padding: 20px; border-radius: 10px; color: white;'>
        <h2 style='margin:0'>🏆 TOP RANKED PLAYS: {scenario['interval']} YDS vs {scenario['defense']}</h2>
        <p>Sorted by <b>Simulated Reliability</b> (10,000 Monte Carlo Trials)</p>
    </div>
    """))

    for i, rec in enumerate(recs):
        rank = i + 1
        
        # Color code reliability
        rel_color = "#2ecc71" if rec['reliability_score'] > 30 else "#f1c40f"
        
        display(HTML(f"""
        <div style='border: 1px solid #ddd; margin-top: 15px; border-radius: 8px; overflow: hidden; box-shadow: 0 4px 6px rgba(0,0,0,0.1);'>
            <div style='background-color: #f8f9fa; padding: 15px; border-left: 6px solid {rel_color}; display: flex; justify-content: space-between; align-items: center;'>
                <div>
                    <h3 style='margin: 0; color: #333;'>#{rank} {rec['formation']} + {rec['route']}</h3>
                </div>
                <div style='text-align: right;'>
                    <span style='font-size: 24px; font-weight: bold; color: {rel_color};'>{rec['simulated_success']}%</span>
                    <br><span style='font-size: 11px; color: #666;'>Simulated Success Rate</span>
                </div>
            </div>
            <div style='padding: 15px; display: flex; justify-content: space-around; background-color: white;'>
                <div style='text-align: center;'>
                    <div style='font-weight: bold; font-size: 18px;'>{rec['reliability_score']}%</div>
                    <div style='font-size: 12px; color: #888;'>Reliability Floor<br>(90% Confidence)</div>
                </div>
                <div style='text-align: center; border-left: 1px solid #eee; padding-left: 20px;'>
                    <div style='font-weight: bold; font-size: 18px;'>{rec['raw_success_rate']}%</div>
                    <div style='font-size: 12px; color: #888;'>Raw Stats<br>({rec['td_count']}/{rec['attempts']})</div>
                </div>
                 <div style='text-align: center; border-left: 1px solid #eee; padding-left: 20px;'>
                    <div style='font-weight: bold; font-size: 18px;'>High</div>
                    <div style='font-size: 12px; color: #888;'>Sample Size<br>Confidence</div>
                </div>
            </div>
        </div>
        """))

# --- WIDGET SETUP ---
yards_slider_rank = widgets.IntSlider(value=12, min=5, max=20, description='Yards Out:')
defense_dropdown_rank = widgets.Dropdown(options=['Man', 'Zone'], value='Man', description='Defense:')
btn_rank = widgets.Button(description='Simulate & Rank Plays', button_style='danger')
out_rank = widgets.Output()

def on_click_rank(b):
    with out_rank:
        clear_output()
        print("🎲 Running 10,000 Monte Carlo simulations per play type...")
        res = get_ranked_recommendations(yards_slider_rank.value, defense_dropdown_rank.value)
        display_ranked_results(res)

btn_rank.on_click(on_click_rank)

# Display
display(widgets.VBox([
    widgets.HTML("<h3>🚀 Monte Carlo Play Ranker</h3>"),
    yards_slider_rank, 
    defense_dropdown_rank, 
    btn_rank, 
    out_rank
]))


In [164]:
# CELL 17: Enhanced Recommendation Engine with Receiver Execution Stats

def get_enhanced_recommendations(yards_out, defense_type):
    """
    Get ranked plays with detailed receiver alignment and execution metrics.
    """
    # 1. Determine Interval
    if 5 <= yards_out <= 10:
        interval = '5-10'
    elif 10 < yards_out <= 15:
        interval = '10-15'
    elif 15 < yards_out <= 20:
        interval = '15-20'
    else:
        return {'error': 'Yards must be between 5-20'}

    # 2. Map Coverage
    available_covs = play_summary['team_coverage_man_zone'].unique()
    mapped_coverage = map_coverage_input(defense_type, available_covs)
    
    print(f"📊 Analyzing: {interval} yards vs {mapped_coverage}...")

    # 3. Filter Data
    scenario_plays = successful_plays[
        (successful_plays['redzone_interval'] == interval) &
        (successful_plays['team_coverage_man_zone'] == mapped_coverage)
    ]
    
    all_attempts_scenario = play_summary[
        (play_summary['redzone_interval'] == interval) &
        (play_summary['team_coverage_man_zone'] == mapped_coverage)
    ]
    
    if len(all_attempts_scenario) < 5:
        return {'error': f"Insufficient sample size ({len(all_attempts_scenario)} plays found)."}

    # 4. Group by Formation + Route + Receiver Alignment
    grouped = all_attempts_scenario.groupby([
        'offense_formation', 
        'route_of_targeted_receiver',
        'receiver_alignment'  # NEW
    ]).agg(
        total_attempts=('play_id', 'count'),
        td_count=('is_touchdown', 'sum')
    ).reset_index()

    grouped = grouped[grouped['total_attempts'] >= 2].copy()

    if grouped.empty:
        return {'error': "No recurring play patterns found."}

    # 5. RUN SIMULATIONS & GATHER RECEIVER STATS
    results = []
    global_avg = len(successful_plays) / len(play_summary)
    
    for _, row in grouped.iterrows():
        # Run Monte Carlo
        expected_rate, reliability_score = simulate_play_reliability(
            row['td_count'], 
            row['total_attempts'], 
            global_avg_rate=global_avg
        )
        
        raw_rate = (row['td_count'] / row['total_attempts']) * 100
        
        # Get tracking data for this play pattern
        pattern_filter = (
            (all_attempts_scenario['offense_formation'] == row['offense_formation']) &
            (all_attempts_scenario['route_of_targeted_receiver'] == row['route_of_targeted_receiver']) &
            (all_attempts_scenario['receiver_alignment'] == row['receiver_alignment'])
        )
        
        pattern_plays = all_attempts_scenario[pattern_filter]
        pattern_tracking = redzone_df[
            redzone_df['play_id'].isin(pattern_plays['play_id'])
        ]
        
        # Filter for TARGETED RECEIVER only
        targeted_receiver = pattern_tracking[
            pattern_tracking['player_role'] == 'Targeted Receiver'
        ]
        
        # EXECUTION STATS FOR TARGETED RECEIVER
        if not targeted_receiver.empty:
            # Pre-snap speed: speed at frame 1 (the snap)
            snap_speed = targeted_receiver.groupby('play_id').apply(
                lambda x: x[x['frame'] == 1]['s'].values[0] 
                if len(x[x['frame'] == 1]) > 0 else np.nan
            ).dropna()
            pre_snap_speed = snap_speed.mean() if len(snap_speed) > 0 else None
            
            # Acceleration: mean acceleration across the route
            acceleration = targeted_receiver['a'].mean() if targeted_receiver['a'].notna().any() else None
            
            # Starting position (at snap, frame 1)
            snap_positions = targeted_receiver[targeted_receiver['frame'] == 1]
            start_x = snap_positions['x'].mean() if not snap_positions.empty else None
            start_y = snap_positions['y'].mean() if not snap_positions.empty else None
            
            # Position flexibility: standard deviation of starting positions
            pos_flex_x = snap_positions['x'].std() if not snap_positions.empty else None
            pos_flex_y = snap_positions['y'].std() if not snap_positions.empty else None
        else:
            pre_snap_speed = acceleration = start_x = start_y = pos_flex_x = pos_flex_y = None
        
        results.append({
            'formation': row['offense_formation'],
            'route': row['route_of_targeted_receiver'],
            'alignment': row['receiver_alignment'],  # NEW
            'td_count': row['td_count'],
            'attempts': row['total_attempts'],
            'raw_success_rate': round(raw_rate, 1),
            'simulated_success': expected_rate,
            'reliability_score': reliability_score,
            
            # Receiver execution stats
            'pre_snap_speed': round(pre_snap_speed, 2) if pre_snap_speed else None,
            'acceleration': round(acceleration, 2) if acceleration else None,
            'start_x': round(start_x, 1) if start_x else None,  # Distance from sideline
            'start_y': round(start_y, 1) if start_y else None,  # Distance from LOS
            'position_flex_x': round(pos_flex_x, 2) if pos_flex_x else None,  # Consistency sideline
            'position_flex_y': round(pos_flex_y, 2) if pos_flex_y else None,  # Consistency depth
        })

    # Sort by reliability
    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values('reliability_score', ascending=False)

    return {
        'scenario': {'interval': interval, 'defense': mapped_coverage},
        'data': df_results.head(5).to_dict('records')
    }

print("✅ Enhanced Recommendation Engine loaded!")


✅ Enhanced Recommendation Engine loaded!


In [165]:
# CELL 18: Enhanced Display with Receiver Execution Details

def display_enhanced_results(result):
    if 'error' in result:
        print(f"❌ {result['error']}")
        return

    scenario = result['scenario']
    recs = result['data']
    
    display(HTML(f"""
    <div style='background: linear-gradient(to right, #1a2a6c, #b21f1f, #fdbb2d); padding: 20px; border-radius: 10px; color: white;'>
        <h2 style='margin:0'>🏆 TOP RANKED PLAYS: {scenario['interval']} YDS vs {scenario['defense']}</h2>
        <p>Sorted by Reliability (10,000 Monte Carlo Trials)</p>
    </div>
    """))

    for i, rec in enumerate(recs):
        rank = i + 1
        rel_color = "#2ecc71" if rec['reliability_score'] > 30 else "#f1c40f"
        
        display(HTML(f"""
        <div style='border: 1px solid #ddd; margin-top: 20px; border-radius: 8px; overflow: hidden; box-shadow: 0 4px 6px rgba(0,0,0,0.1);'>
            <div style='background-color: #f8f9fa; padding: 15px; border-left: 6px solid {rel_color};'>
                <h3 style='margin: 0; color: #333;'>#{rank} {rec['formation']} + {rec['route']}</h3>
                <p style='margin: 5px 0 0 0; font-size: 12px; color: #666;'><b>Alignment:</b> {rec['alignment']}</p>
            </div>
            
            <div style='padding: 15px; background-color: white;'>
                <div style='display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 15px; margin-bottom: 15px;'>
                    <div style='text-align: center; border-right: 1px solid #eee;'>
                        <div style='font-weight: bold; font-size: 22px; color: {rel_color};'>{rec['reliability_score']}%</div>
                        <div style='font-size: 11px; color: #888;'>Reliability Score<br>(90% Confidence)</div>
                    </div>
                    <div style='text-align: center; border-right: 1px solid #eee;'>
                        <div style='font-weight: bold; font-size: 18px;'>{rec['simulated_success']}%</div>
                        <div style='font-size: 11px; color: #888;'>Simulated Success<br>({rec['td_count']}/{rec['attempts']} TDs)</div>
                    </div>
                    <div style='text-align: center;'>
                        <div style='font-weight: bold; font-size: 18px;'>{rec['raw_success_rate']}%</div>
                        <div style='font-size: 11px; color: #888;'>Raw Stats</div>
                    </div>
                </div>
                
                <hr style='border: 0; border-top: 1px solid #eee; margin: 15px 0;'>
                
                <h4 style='margin: 0 0 10px 0; color: #333; font-size: 13px;'>🎯 Targeted Receiver Execution Stats</h4>
                
                <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 10px; font-size: 12px;'>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Pre-Snap Speed:</span>
                        <div style='font-weight: bold; color: #333;'>{rec['pre_snap_speed']} yd/s</div>
                    </div>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Acceleration:</span>
                        <div style='font-weight: bold; color: #333;'>{rec['acceleration']} yd/s²</div>
                    </div>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Start X (Sideline):</span>
                        <div style='font-weight: bold; color: #333;'>{rec['start_x']} yd</div>
                    </div>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Start Y (Depth):</span>
                        <div style='font-weight: bold; color: #333;'>{rec['start_y']} yd</div>
                    </div>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Position Flex (X):</span>
                        <div style='font-weight: bold; color: #333;'>±{rec['position_flex_x']} yd</div>
                    </div>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Position Flex (Y):</span>
                        <div style='font-weight: bold; color: #333;'>±{rec['position_flex_y']} yd</div>
                    </div>
                </div>
            </div>
        </div>
        """))

# Widget setup
yards_slider_enh = widgets.IntSlider(value=12, min=5, max=20, description='Yards Out:')
defense_dropdown_enh = widgets.Dropdown(options=['Man', 'Zone'], value='Man', description='Defense:')
btn_enh = widgets.Button(description='Run Enhanced Simulation', button_style='success')
out_enh = widgets.Output()

def on_click_enhanced(b):
    with out_enh:
        clear_output()
        print("🎬 Running enhanced Monte Carlo with receiver stats...")
        res = get_enhanced_recommendations(yards_slider_enh.value, defense_dropdown_enh.value)
        display_enhanced_results(res)

btn_enh.on_click(on_click_enhanced)

display(widgets.VBox([
    widgets.HTML("<h3>🚀 Enhanced Monte Carlo Play Ranker with Receiver Execution</h3>"),
    yards_slider_enh, 
    defense_dropdown_enh, 
    btn_enh, 
    out_enh
]))

print("✅ Enhanced UI loaded!")


✅ Enhanced UI loaded!


In [166]:
# CELL 21: Calculate Speed & Acceleration from Tracking Data

def calculate_receiver_kinematics(tracking_data):
    """
    Calculate speed and acceleration from raw x,y position data.
    
    Args:
        tracking_data: DataFrame with columns [frame_id, x, y, ...]
    
    Returns:
        dict with 'speed', 'acceleration', 'start_x', 'start_y', 'pos_flex_x', 'pos_flex_y'
    """
    
    if tracking_data.empty:
        return {
            'avg_speed': None,
            'max_speed': None,
            'avg_accel': None,
            'start_x': None,
            'start_y': None,
            'pos_flex_x': None,
            'pos_flex_y': None,
        }
    
    # Sort by frame to ensure correct order
    tracking_data = tracking_data.sort_values('frame_id')
    
    # Extract positions
    frames = tracking_data['frame_id'].values
    x = tracking_data['x'].values
    y = tracking_data['y'].values
    
    # Distance between frames (assuming 10 frames per second in NFL)
    # Each frame = 0.1 seconds
    frame_interval = 0.1  # seconds
    
    # Calculate distance traveled per frame
    dx = np.diff(x)
    dy = np.diff(y)
    distance_per_frame = np.sqrt(dx**2 + dy**2)  # Euclidean distance
    
    # Speed = distance / time (yards per second)
    # NFL tracking data is in yards
    speed_per_frame = distance_per_frame / frame_interval
    
    # Acceleration = change in speed / time (yards per second²)
    dv = np.diff(speed_per_frame)
    acceleration_per_frame = dv / frame_interval
    
    # Aggregate stats
    avg_speed = np.mean(speed_per_frame) if len(speed_per_frame) > 0 else None
    max_speed = np.max(speed_per_frame) if len(speed_per_frame) > 0 else None
    avg_accel = np.mean(np.abs(acceleration_per_frame)) if len(acceleration_per_frame) > 0 else None
    
    # Starting position (frame 1)
    start_x = x[0] if len(x) > 0 else None
    start_y = y[0] if len(y) > 0 else None
    
    # Position consistency across multiple plays (will compute in loop)
    # These will be filled in by grouping across plays
    
    return {
        'avg_speed': avg_speed,
        'max_speed': max_speed,
        'avg_accel': avg_accel,
        'start_x': start_x,
        'start_y': start_y,
    }

print("✅ Kinematics calculator loaded!")


✅ Kinematics calculator loaded!


In [167]:
# CELL 22: Enhanced Recommendations with Kinematics

def get_enhanced_recommendations_fixed(yards_out, defense_type):
    """
    Get ranked plays with receiver execution stats calculated from position data.
    """
    
    # 1. Determine Interval
    if 5 <= yards_out <= 10:
        interval = '5-10'
    elif 10 < yards_out <= 15:
        interval = '10-15'
    elif 15 < yards_out <= 20:
        interval = '15-20'
    else:
        return {'error': 'Yards must be between 5-20'}

    # 2. Map Coverage
    available_covs = play_summary['team_coverage_man_zone'].unique()
    mapped_coverage = map_coverage_input(defense_type, available_covs)
    
    print(f"📊 Analyzing: {interval} yards vs {mapped_coverage}...")

    # 3. Filter Data
    scenario_plays = successful_plays[
        (successful_plays['redzone_interval'] == interval) &
        (successful_plays['team_coverage_man_zone'] == mapped_coverage)
    ]
    
    all_attempts_scenario = play_summary[
        (play_summary['redzone_interval'] == interval) &
        (play_summary['team_coverage_man_zone'] == mapped_coverage)
    ]
    
    if len(all_attempts_scenario) < 5:
        return {'error': f"Insufficient sample size ({len(all_attempts_scenario)} plays)."}

    # 4. Group by Formation + Route + Receiver Alignment
    grouped = all_attempts_scenario.groupby([
        'offense_formation', 
        'route_of_targeted_receiver',
        'receiver_alignment'
    ]).agg(
        total_attempts=('play_id', 'count'),
        td_count=('is_touchdown', 'sum')
    ).reset_index()

    grouped = grouped[grouped['total_attempts'] >= 2].copy()

    if grouped.empty:
        return {'error': "No recurring patterns found."}

    # 5. RUN SIMULATIONS & CALCULATE KINEMATICS
    results = []
    global_avg = len(successful_plays) / len(play_summary)
    
    for _, row in grouped.iterrows():
        # Run Monte Carlo
        expected_rate, reliability_score = simulate_play_reliability(
            row['td_count'], 
            row['total_attempts'], 
            global_avg_rate=global_avg
        )
        
        raw_rate = (row['td_count'] / row['total_attempts']) * 100
        
        # Get TD plays matching this pattern
        td_plays = scenario_plays[
            (scenario_plays['offense_formation'] == row['offense_formation']) &
            (scenario_plays['route_of_targeted_receiver'] == row['route_of_targeted_receiver']) &
            (scenario_plays['receiver_alignment'] == row['receiver_alignment'])
        ]
        
        # Get tracking data for these TD plays
        # IMPORTANT: We need to identify the "targeted receiver" by position/jersey number
        # Since there's no 'player_role' column, we'll use the player from play_summary
        
        play_ids = td_plays['play_id'].values
        pattern_tracking = redzone_df[redzone_df['play_id'].isin(play_ids)]
        
        # Get tracking for the targeted receiver
        # We need to know which player was the target — let's assume it's the one with most receiving yards
        # For now, calculate stats for ALL receivers in successful plays and average them
        
        # Better approach: Get the receiver who had the highest x displacement (got deepest)
        receiver_stats_list = []
        
        for play_id in play_ids:
            play_tracking = pattern_tracking[pattern_tracking['play_id'] == play_id]
            
            # Get all unique players in this play
            for player in play_tracking['player_to_predict'].unique():
                player_tracking = play_tracking[play_tracking['player_to_predict'] == player]
                
                # Only look at offensive players (we can filter by position if needed)
                # For now, assume receivers move more than O-line
                if len(player_tracking) > 1:  # Only if we have multiple frames
                    kinematics = calculate_receiver_kinematics(player_tracking)
                    receiver_stats_list.append(kinematics)
        
        # Average across all receiver stats in this play pattern
        if receiver_stats_list:
            avg_speed = np.nanmean([r['avg_speed'] for r in receiver_stats_list if r['avg_speed'] is not None]) if receiver_stats_list else None
            max_speed = np.nanmean([r['max_speed'] for r in receiver_stats_list if r['max_speed'] is not None]) if receiver_stats_list else None
            avg_accel = np.nanmean([r['avg_accel'] for r in receiver_stats_list if r['avg_accel'] is not None]) if receiver_stats_list else None
            
            # Position consistency
            start_x_vals = [r['start_x'] for r in receiver_stats_list if r['start_x'] is not None]
            start_y_vals = [r['start_y'] for r in receiver_stats_list if r['start_y'] is not None]
            
            start_x = np.mean(start_x_vals) if start_x_vals else None
            start_y = np.mean(start_y_vals) if start_y_vals else None
            pos_flex_x = np.std(start_x_vals) if len(start_x_vals) > 1 else None
            pos_flex_y = np.std(start_y_vals) if len(start_y_vals) > 1 else None
        else:
            avg_speed = max_speed = avg_accel = start_x = start_y = pos_flex_x = pos_flex_y = None
        
        results.append({
            'formation': row['offense_formation'],
            'route': row['route_of_targeted_receiver'],
            'alignment': row['receiver_alignment'],
            'td_count': row['td_count'],
            'attempts': row['total_attempts'],
            'raw_success_rate': round(raw_rate, 1),
            'simulated_success': expected_rate,
            'reliability_score': reliability_score,
            
            # Calculated kinematics
            'avg_speed': round(avg_speed, 2) if avg_speed else None,
            'max_speed': round(max_speed, 2) if max_speed else None,
            'avg_accel': round(avg_accel, 2) if avg_accel else None,
            'start_x': round(start_x, 1) if start_x else None,
            'start_y': round(start_y, 1) if start_y else None,
            'pos_flex_x': round(pos_flex_x, 2) if pos_flex_x else None,
            'pos_flex_y': round(pos_flex_y, 2) if pos_flex_y else None,
        })

    # Sort by reliability
    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values('reliability_score', ascending=False)

    return {
        'scenario': {'interval': interval, 'defense': mapped_coverage},
        'data': df_results.head(5).to_dict('records')
    }

print("✅ Enhanced engine (fixed) loaded!")


✅ Enhanced engine (fixed) loaded!


In [ ]:
# CELL 23: Widget Setup

yards_slider_fixed = widgets.IntSlider(value=12, min=5, max=20, description='Yards Out:')
defense_dropdown_fixed = widgets.Dropdown(options=['Man', 'Zone'], value='Man', description='Defense:')
btn_fixed = widgets.Button(description='Run Enhanced Simulation', button_style='success')
out_fixed = widgets.Output()

def on_click_fixed(b):
    with out_fixed:
        clear_output()
        print("🎬 Running Monte Carlo with calculated receiver kinematics...")
        res = get_enhanced_recommendations_fixed(yards_slider_fixed.value, defense_dropdown_fixed.value)
        display_enhanced_results(res)

btn_fixed.on_click(on_click_fixed)

display(widgets.VBox([
    widgets.HTML("<h3>🚀 Monte Carlo Play Ranker (Receiver Kinematics)</h3>"),
    yards_slider_fixed, 
    defense_dropdown_fixed, 
    btn_fixed, 
    out_fixed
]))

print("✅ Widget ready!")


✅ Widget ready!


In [169]:
# CELL 24: Bulletproof Display Function

def display_enhanced_results_v2(result):
    """
    Display results with None-safe formatting.
    """
    if 'error' in result:
        display(HTML(f"<h3 style='color: red;'>❌ {result['error']}</h3>"))
        return

    scenario = result['scenario']
    recs = result['data']
    
    if not recs or len(recs) == 0:
        display(HTML("<h3 style='color: red;'>❌ No plays found.</h3>"))
        return
    
    display(HTML(f"""
    <div style='background: linear-gradient(to right, #1a2a6c, #b21f1f, #fdbb2d); padding: 20px; border-radius: 10px; color: white;'>
        <h2 style='margin:0'>🏆 TOP RANKED PLAYS: {scenario['interval']} YDS vs {scenario['defense']}</h2>
        <p>Sorted by Reliability (Monte Carlo)</p>
    </div>
    """))

    for i, rec in enumerate(recs):
        rank = i + 1
        
        # Safe access with defaults
        formation = rec.get('formation', 'Unknown')
        route = rec.get('route', 'Unknown')
        alignment = rec.get('alignment', 'Unknown')
        reliability = rec.get('reliability_score', 0)
        simulated = rec.get('simulated_success', 0)
        raw = rec.get('raw_success_rate', 0)
        td_count = rec.get('td_count', 0)
        attempts = rec.get('attempts', 0)
        
        # Kinematics (can be None)
        avg_speed = rec.get('avg_speed', 'N/A')
        max_speed = rec.get('max_speed', 'N/A')
        avg_accel = rec.get('avg_accel', 'N/A')
        start_x = rec.get('start_x', 'N/A')
        start_y = rec.get('start_y', 'N/A')
        pos_flex_x = rec.get('pos_flex_x', 'N/A')
        pos_flex_y = rec.get('pos_flex_y', 'N/A')
        
        rel_color = "#2ecc71" if reliability > 30 else "#f1c40f"
        
        display(HTML(f"""
        <div style='border: 1px solid #ddd; margin-top: 20px; border-radius: 8px; overflow: hidden; box-shadow: 0 4px 6px rgba(0,0,0,0.1);'>
            <div style='background-color: #f8f9fa; padding: 15px; border-left: 6px solid {rel_color};'>
                <h3 style='margin: 0; color: #333;'>#{rank} {formation} + {route}</h3>
                <p style='margin: 5px 0 0 0; font-size: 12px; color: #666;'><b>Alignment:</b> {alignment}</p>
            </div>
            
            <div style='padding: 15px; background-color: white;'>
                <div style='display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 15px; margin-bottom: 15px;'>
                    <div style='text-align: center; border-right: 1px solid #eee;'>
                        <div style='font-weight: bold; font-size: 22px; color: {rel_color};'>{reliability}%</div>
                        <div style='font-size: 11px; color: #888;'>Reliability Score<br>(90% Confidence)</div>
                    </div>
                    <div style='text-align: center; border-right: 1px solid #eee;'>
                        <div style='font-weight: bold; font-size: 18px;'>{simulated}%</div>
                        <div style='font-size: 11px; color: #888;'>Simulated Success<br>({td_count}/{attempts} TDs)</div>
                    </div>
                    <div style='text-align: center;'>
                        <div style='font-weight: bold; font-size: 18px;'>{raw}%</div>
                        <div style='font-size: 11px; color: #888;'>Raw Stats</div>
                    </div>
                </div>
                
                <hr style='border: 0; border-top: 1px solid #eee; margin: 15px 0;'>
                
                <h4 style='margin: 0 0 10px 0; color: #333; font-size: 13px;'>🎯 Receiver Kinematics</h4>
                
                <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 10px; font-size: 12px;'>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Avg Speed:</span>
                        <div style='font-weight: bold; color: #333;'>{avg_speed} yd/s</div>
                    </div>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Max Speed:</span>
                        <div style='font-weight: bold; color: #333;'>{max_speed} yd/s</div>
                    </div>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Acceleration:</span>
                        <div style='font-weight: bold; color: #333;'>{avg_accel} yd/s²</div>
                    </div>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Starting X:</span>
                        <div style='font-weight: bold; color: #333;'>{start_x} yd</div>
                    </div>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Starting Y:</span>
                        <div style='font-weight: bold; color: #333;'>{start_y} yd</div>
                    </div>
                    <div style='background: #f5f5f5; padding: 10px; border-radius: 6px;'>
                        <span style='color: #666;'>Position Flex (X±Y):</span>
                        <div style='font-weight: bold; color: #333;'>±{pos_flex_x}±{pos_flex_y} yd</div>
                    </div>
                </div>
            </div>
        </div>
        """))

print("✅ Fixed display function loaded!")


✅ Fixed display function loaded!


In [170]:
# CELL 25: Test with Debugging

yards_slider_test = widgets.IntSlider(value=12, min=5, max=20, description='Yards Out:')
defense_dropdown_test = widgets.Dropdown(options=['Man', 'Zone'], value='Man', description='Defense:')
btn_test = widgets.Button(description='Test Enhanced Simulation', button_style='info')
out_test = widgets.Output()

def on_click_test(b):
    with out_test:
        clear_output()
        try:
            yards = yards_slider_test.value
            defense = defense_dropdown_test.value
            print(f"🎬 Running for {yards} yards vs {defense}...")
            
            res = get_enhanced_recommendations_fixed(yards, defense)
            
            print(f"✅ Got result with keys: {res.keys()}")
            if 'data' in res:
                print(f"✅ Number of plays: {len(res['data'])}")
                if res['data']:
                    print(f"✅ First play keys: {res['data'][0].keys()}")
            
            display_enhanced_results_v2(res)
        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()

btn_test.on_click(on_click_test)

display(widgets.VBox([
    widgets.HTML("<h3>🚀 Test Enhanced Simulation</h3>"),
    yards_slider_test, 
    defense_dropdown_test, 
    btn_test, 
    out_test
]))

print("✅ Test widget ready!")


✅ Test widget ready!


In [189]:
# CELL 1: Speed Effort Conversion Engine (CORRECTED)

FORTY_YARD_DASH_TIME = 4.49
FORTY_YARDS = 40

MAX_WR_SPEED_YDS_PER_SEC = FORTY_YARDS / FORTY_YARD_DASH_TIME

SPEED_SCALE_FACTOR = 3

print(f"📊 Max WR Speed: {MAX_WR_SPEED_YDS_PER_SEC:.3f} yd/s (from 4.49s 40-yard dash)")
print(f"📏 Speed scale factor: {SPEED_SCALE_FACTOR}x (tracking data correction)")

def calculate_speed_effort_pct(speed_yd_per_sec, max_speed=MAX_WR_SPEED_YDS_PER_SEC):
    if max_speed == 0:
        return 0
    scaled_speed = speed_yd_per_sec / SPEED_SCALE_FACTOR
    effort_pct = (scaled_speed / max_speed) * 100
    effort_pct = min(effort_pct, 110)
    effort_pct = max(effort_pct, 0)
    return round(effort_pct, 1)

def calculate_accel_effort_pct(acceleration_yd_per_sec2):
    ELITE_ACCEL = 3.75
    scaled_accel = acceleration_yd_per_sec2 / SPEED_SCALE_FACTOR
    accel_pct = (scaled_accel / ELITE_ACCEL) * 100
    accel_pct = min(accel_pct, 120)
    accel_pct = max(accel_pct, 0)
    return round(accel_pct, 1)

print("✅ Speed effort conversion loaded!")


📊 Max WR Speed: 8.909 yd/s (from 4.49s 40-yard dash)
📏 Speed scale factor: 3x (tracking data correction)
✅ Speed effort conversion loaded!


In [190]:
# CELL 27: Enhanced Kinematics with Effort %

def calculate_receiver_kinematics_with_effort(tracking_data):
    """
    Calculate speed, acceleration, AND effort % from position data.
    """
    
    if tracking_data.empty:
        return {
            'avg_speed': None,
            'max_speed': None,
            'avg_effort_pct': None,
            'max_effort_pct': None,
            'avg_accel': None,
            'avg_accel_effort_pct': None,
            'start_x': None,
            'start_y': None,
        }
    
    tracking_data = tracking_data.sort_values('frame_id')
    
    frames = tracking_data['frame_id'].values
    x = tracking_data['x'].values
    y = tracking_data['y'].values
    
    frame_interval = 0.1  # 10 fps
    
    # Calculate velocities
    dx = np.diff(x)
    dy = np.diff(y)
    distance_per_frame = np.sqrt(dx**2 + dy**2)
    speed_per_frame = distance_per_frame / frame_interval
    
    # Convert to effort %
    effort_pct_per_frame = [calculate_speed_effort_pct(s) for s in speed_per_frame]
    
    # Acceleration
    dv = np.diff(speed_per_frame)
    acceleration_per_frame = dv / frame_interval
    accel_effort_pct_per_frame = [calculate_accel_effort_pct(a) for a in acceleration_per_frame]
    
    # Aggregate stats
    avg_speed = np.mean(speed_per_frame) if len(speed_per_frame) > 0 else None
    max_speed = np.max(speed_per_frame) if len(speed_per_frame) > 0 else None
    avg_effort_pct = np.mean(effort_pct_per_frame) if effort_pct_per_frame else None
    max_effort_pct = np.max(effort_pct_per_frame) if effort_pct_per_frame else None
    
    avg_accel = np.mean(np.abs(acceleration_per_frame)) if len(acceleration_per_frame) > 0 else None
    avg_accel_effort_pct = np.mean(accel_effort_pct_per_frame) if accel_effort_pct_per_frame else None
    
    # Position
    start_x = x[0] if len(x) > 0 else None
    start_y = y[0] if len(y) > 0 else None
    
    return {
        'avg_speed': round(avg_speed, 2) if avg_speed else None,
        'max_speed': round(max_speed, 2) if max_speed else None,
        'avg_effort_pct': avg_effort_pct,
        'max_effort_pct': max_effort_pct,
        'avg_accel': round(avg_accel, 2) if avg_accel else None,
        'avg_accel_effort_pct': avg_accel_effort_pct,
        'start_x': start_x,
        'start_y': start_y,
    }

print("✅ Kinematics calculator with effort % loaded!")


✅ Kinematics calculator with effort % loaded!


In [ ]:
# CELL 28: Display Recommendations with Effort %

def display_enhanced_results_with_effort(result):
    """Display results with speed effort % prominently featured."""
    
    if 'error' in result:
        display(HTML(f"<h3 style='color: red;'>❌ {result['error']}</h3>"))
        return

    scenario = result['scenario']
    recs = result['data']
    
    if not recs or len(recs) == 0:
        display(HTML("<h3 style='color: red;'>❌ No plays found.</h3>"))
        return
    
    display(HTML(f"""
    <div style='background: linear-gradient(to right, #1a2a6c, #b21f1f, #fdbb2d); padding: 20px; border-radius: 10px; color: white;'>
        <h2 style='margin:0'>🏆 TOP RANKED PLAYS: {scenario['interval']} YDS vs {scenario['defense']}</h2>
        <p>Baseline: WR 40-yard dash = {FORTY_YARD_DASH_TIME}s | Max Speed = {MAX_WR_SPEED_YDS_PER_SEC:.2f} yd/s</p>
    </div>
    """))

    for i, rec in enumerate(recs):
        rank = i + 1
        rel_color = "#2ecc71" if rec.get('reliability_score', 0) > 30 else "#f1c40f"
        
        # Safe access
        avg_effort = rec.get('avg_effort_pct', 'N/A')
        max_effort = rec.get('max_effort_pct', 'N/A')
        accel_effort = rec.get('avg_accel_effort_pct', 'N/A')
        
        display(HTML(f"""
        <div style='border: 1px solid #ddd; margin-top: 20px; border-radius: 8px; overflow: hidden; box-shadow: 0 4px 6px rgba(0,0,0,0.1);'>
            <div style='background-color: #f8f9fa; padding: 15px; border-left: 6px solid {rel_color};'>
                <h3 style='margin: 0; color: #333;'>#{rank} {rec.get('formation', 'Unknown')} + {rec.get('route', 'Unknown')}</h3>
                <p style='margin: 5px 0 0 0; font-size: 12px; color: #666;'><b>Alignment:</b> {rec.get('alignment', 'Unknown')}</p>
            </div>
            
            <div style='padding: 15px; background-color: white;'>
                <div style='display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 15px; margin-bottom: 15px;'>
                    <div style='text-align: center; border-right: 1px solid #eee;'>
                        <div style='font-weight: bold; font-size: 22px; color: {rel_color};'>{rec.get('reliability_score', 0)}%</div>
                        <div style='font-size: 11px; color: #888;'>Reliability<br>(90% Confidence)</div>
                    </div>
                    <div style='text-align: center; border-right: 1px solid #eee;'>
                        <div style='font-weight: bold; font-size: 18px;'>{rec.get('simulated_success', 0)}%</div>
                        <div style='font-size: 11px; color: #888;'>Success Rate<br>({rec.get('td_count', 0)}/{rec.get('attempts', 0)})</div>
                    </div>
                    <div style='text-align: center;'>
                        <div style='font-weight: bold; font-size: 18px;'>{rec.get('raw_success_rate', 0)}%</div>
                        <div style='font-size: 11px; color: #888;'>Raw Stats</div>
                    </div>
                </div>
                
                <hr style='border: 0; border-top: 1px solid #eee; margin: 15px 0;'>
                
                <h4 style='margin: 0 0 10px 0; color: #333; font-size: 13px;'>⚡ Speed Effort (% of Max WR Capability)</h4>
                
                <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 10px; font-size: 12px; margin-bottom: 15px;'>
                    <div style='background: linear-gradient(to right, #fff3cd, #ffc107); padding: 12px; border-radius: 6px; border: 1px solid #ffc107;'>
                        <span style='color: #333; font-weight: bold;'>📊 Avg Speed Effort</span>
                        <div style='font-size: 20px; font-weight: bold; color: #ff6b00;'>{avg_effort}%</div>
                        <div style='font-size: 10px; color: #666; margin-top: 5px;'>Expected pre-snap/route</div>
                    </div>
                    <div style='background: linear-gradient(to right, #ffcccc, #ff6666); padding: 12px; border-radius: 6px; border: 1px solid #ff6666;'>
                        <span style='color: #333; font-weight: bold;'>🔥 Max Speed Effort</span>
                        <div style='font-size: 20px; font-weight: bold; color: #cc0000;'>{max_effort}%</div>
                        <div style='font-size: 10px; color: #666; margin-top: 5px;'>Peak explosion moment</div>
                    </div>
                </div>
                
                <h4 style='margin: 0 0 10px 0; color: #333; font-size: 13px;'>💨 Acceleration Effort</h4>
                
                <div style='background: #e8f5e9; padding: 12px; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <div style='font-weight: bold; font-size: 18px; color: #2e7d32;'>{accel_effort}%</div>
                    <div style='font-size: 11px; color: #666; margin-top: 5px;'>Acceleration % effort (elite baseline: 3.75 yd/s²)</div>
                </div>
            </div>
        </div>
        """))

# Widget
yards_slider_effort = widgets.IntSlider(value=12, min=5, max=20, description='Yards Out:')
defense_dropdown_effort = widgets.Dropdown(options=['Man', 'Zone'], value='Man', description='Defense:')
btn_effort = widgets.Button(description='Analyze with Effort %', button_style='info')
out_effort = widgets.Output()

def on_click_effort(b):
    with out_effort:
        clear_output()
        print("⚡ Calculating optimal speed effort %...")
        res = get_enhanced_recommendations_fixed(yards_slider_effort.value, defense_dropdown_effort.value)
        display_enhanced_results_with_effort(res)

btn_effort.on_click(on_click_effort)

display(widgets.VBox([
    widgets.HTML("<h3>⚡ Red Zone Speed Effort Analyzer</h3>"),
    yards_slider_effort,
    defense_dropdown_effort,
    btn_effort,
    out_effort
]))

print("✅ Effort % widget ready!")


✅ Effort % widget ready!


In [192]:
# CELL 29: Fix - Updated Recommendations with Effort % Calculation

def get_enhanced_recommendations_fixed(yards_out, defense_type):
    """
    Get ranked plays with receiver kinematics INCLUDING effort %.
    """
    
    # 1. Determine Interval
    if 5 <= yards_out <= 10:
        interval = '5-10'
    elif 10 < yards_out <= 15:
        interval = '10-15'
    elif 15 < yards_out <= 20:
        interval = '15-20'
    else:
        return {'error': 'Yards must be between 5-20'}

    # 2. Map Coverage
    available_covs = play_summary['team_coverage_man_zone'].unique()
    mapped_coverage = map_coverage_input(defense_type, available_covs)
    
    print(f"📊 Analyzing: {interval} yards vs {mapped_coverage}...")

    # 3. Filter Data
    scenario_plays = successful_plays[
        (successful_plays['redzone_interval'] == interval) &
        (successful_plays['team_coverage_man_zone'] == mapped_coverage)
    ]
    
    all_attempts_scenario = play_summary[
        (play_summary['redzone_interval'] == interval) &
        (play_summary['team_coverage_man_zone'] == mapped_coverage)
    ]
    
    if len(all_attempts_scenario) < 5:
        return {'error': f"Insufficient sample size ({len(all_attempts_scenario)} plays)."}

    # 4. Group by Formation + Route + Receiver Alignment
    grouped = all_attempts_scenario.groupby([
        'offense_formation', 
        'route_of_targeted_receiver',
        'receiver_alignment'
    ]).agg(
        total_attempts=('play_id', 'count'),
        td_count=('is_touchdown', 'sum')
    ).reset_index()

    grouped = grouped[grouped['total_attempts'] >= 2].copy()

    if grouped.empty:
        return {'error': "No recurring patterns found."}

    # 5. RUN SIMULATIONS & CALCULATE KINEMATICS WITH EFFORT %
    results = []
    global_avg = len(successful_plays) / len(play_summary)
    
    for _, row in grouped.iterrows():
        # Run Monte Carlo
        expected_rate, reliability_score = simulate_play_reliability(
            row['td_count'], 
            row['total_attempts'], 
            global_avg_rate=global_avg
        )
        
        raw_rate = (row['td_count'] / row['total_attempts']) * 100
        
        # Get TD plays matching this pattern
        td_plays = scenario_plays[
            (scenario_plays['offense_formation'] == row['offense_formation']) &
            (scenario_plays['route_of_targeted_receiver'] == row['route_of_targeted_receiver']) &
            (scenario_plays['receiver_alignment'] == row['receiver_alignment'])
        ]
        
        # Get tracking data for these TD plays
        play_ids = td_plays['play_id'].values
        pattern_tracking = redzone_df[redzone_df['play_id'].isin(play_ids)]
        
        # Get tracking for all receivers in successful plays and calculate kinematics
        receiver_stats_list = []
        
        for play_id in play_ids:
            play_tracking = pattern_tracking[pattern_tracking['play_id'] == play_id]
            
            # Get all unique players in this play
            for player in play_tracking['player_to_predict'].unique():
                player_tracking = play_tracking[play_tracking['player_to_predict'] == player]
                
                # Only if we have multiple frames
                if len(player_tracking) > 1:
                    # USE THE NEW FUNCTION WITH EFFORT %
                    kinematics = calculate_receiver_kinematics_with_effort(player_tracking)
                    receiver_stats_list.append(kinematics)
        
        # Average across all receiver stats in this play pattern
        if receiver_stats_list:
            avg_speed = np.nanmean([r['avg_speed'] for r in receiver_stats_list if r['avg_speed'] is not None]) if receiver_stats_list else None
            max_speed = np.nanmean([r['max_speed'] for r in receiver_stats_list if r['max_speed'] is not None]) if receiver_stats_list else None
            
            # EFFORT % STATS
            avg_effort_pct = np.nanmean([r['avg_effort_pct'] for r in receiver_stats_list if r['avg_effort_pct'] is not None]) if receiver_stats_list else None
            max_effort_pct = np.nanmean([r['max_effort_pct'] for r in receiver_stats_list if r['max_effort_pct'] is not None]) if receiver_stats_list else None
            avg_accel_effort_pct = np.nanmean([r['avg_accel_effort_pct'] for r in receiver_stats_list if r['avg_accel_effort_pct'] is not None]) if receiver_stats_list else None
            
            avg_accel = np.nanmean([r['avg_accel'] for r in receiver_stats_list if r['avg_accel'] is not None]) if receiver_stats_list else None
            
            # Position consistency
            start_x_vals = [r['start_x'] for r in receiver_stats_list if r['start_x'] is not None]
            start_y_vals = [r['start_y'] for r in receiver_stats_list if r['start_y'] is not None]
            
            start_x = np.mean(start_x_vals) if start_x_vals else None
            start_y = np.mean(start_y_vals) if start_y_vals else None
            pos_flex_x = np.std(start_x_vals) if len(start_x_vals) > 1 else None
            pos_flex_y = np.std(start_y_vals) if len(start_y_vals) > 1 else None
        else:
            avg_speed = max_speed = avg_effort_pct = max_effort_pct = avg_accel_effort_pct = avg_accel = start_x = start_y = pos_flex_x = pos_flex_y = None
        
        results.append({
            'formation': row['offense_formation'],
            'route': row['route_of_targeted_receiver'],
            'alignment': row['receiver_alignment'],
            'td_count': row['td_count'],
            'attempts': row['total_attempts'],
            'raw_success_rate': round(raw_rate, 1),
            'simulated_success': expected_rate,
            'reliability_score': reliability_score,
            
            # Raw kinematics
            'avg_speed': avg_speed,
            'max_speed': max_speed,
            'avg_accel': avg_accel,
            
            # EFFORT % (NEW)
            'avg_effort_pct': avg_effort_pct,
            'max_effort_pct': max_effort_pct,
            'avg_accel_effort_pct': avg_accel_effort_pct,
            
            # Position
            'start_x': round(start_x, 1) if start_x else None,
            'start_y': round(start_y, 1) if start_y else None,
            'pos_flex_x': round(pos_flex_x, 2) if pos_flex_x else None,
            'pos_flex_y': round(pos_flex_y, 2) if pos_flex_y else None,
        })

    # Sort by reliability
    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values('reliability_score', ascending=False)

    return {
        'scenario': {'interval': interval, 'defense': mapped_coverage},
        'data': df_results.head(5).to_dict('records')
    }

print("✅ Fixed recommendation engine with effort % loaded!")


✅ Fixed recommendation engine with effort % loaded!
